# Trabalho prático 3

**Grupo 5**:
* Duarte Oliveira \<pg47157\>
* Melânia Pereira \<pg47520\>

# Rainbow 
Criação de um protótipo em Sagemath do algoritmo Rainbow, candidato ao [concurso NIST Post-Quantum Cryptography](https://csrc.nist.gov/Projects/post-quantum-cryptography/round-3-submissions) na categoria de esquemas de assinatura digital.

# Inicialização

In [447]:
import hashlib

In [448]:
class RAIN:
    #Geração dos parâmetros (Security Category V)
    def __init__(self): 
        self.q = 256                                        
        self.v1 = 96
        self.o1 = 36
        self.o2 = 64
        
        F.<x> = GF(self.q)
        self.F = F

In [449]:
class RAIN(RAIN):
    
    #Algoritmo de geração de chaves  
    def keygen(self):
        m = self.o1 + self.o2
        n = m + self.v1

        Ms = matrix.random(self.F, m, m)

        while not Ms.is_invertible():
            Ms = matrix.random(self.F, m, m)
            
        cs = self.genC(m)

        s = self.aff(Ms,cs)

        invS = Ms.inverse_of_unit()

        Mt = matrix.random(self.F, n, n)

        while not Mt.is_invertible():
            Mt = matrix.random(self.F, n, n)
        
        ct = self.genC(n)
        t = self.aff(Mt,ct)

        invT = Mt.inverse_of_unit()

        f = self.rainbowMap()

        p = compose(compose(s,f),t)

        sk = (invS,cs,f,invT,ct)
        pk = p

        return (sk,pk)

    def genC(self,tam): # o que é o c? um escalar ou um vetor???
        c = []
        for i in range(tam):
            c.append(self.F.random_element())
        return vector(self.F,c)

    def aff(self,M,c):
        A = AffineGroup(len(M[0]),self.F)

        return A(M,c)

    def rainbowMap(self): #not sure se isto está certo (vi num dos videos)
        m = self.o1 + self.o2
        n = m + self.v1
        
        A = Matrix.random(self.F, n, n)
        B = Matrix.random(self.F, n, n)
        C = Matrix.random(self.F, n, n)

        E1 = A*B
        E2 = A*C 

        S = []
        for i in range(n):
            S.append([])
            for j in range(n):
                S[i].append(E1[i][j])
        for i in range(n):
            S.append([])
            for j in range(n):
                S[i+n].append(E2[i][j])
        
        return S

        

In [453]:
class RAIN(RAIN):
    
    #Algoritmo de assinatura  
    def sign(self,sk,d):
        (invS,cs,f,invT,ct) = sk

        h = hashlib.shake_256(d.encode()).digest(int(32))

        x = invS * (h - cs) #errror

        y = self.invF(f,x) 

        z = invT * (y - ct)

        return z


    def invF(self,f,x):
        #TODO
        return 0

In [451]:
class RAIN(RAIN):
    
    #Algoritmo de verificação  
    def ver(self,d,z,pk):
        h = hashlib.shake_256(d.encode()).digest(int(32))

        h_ = pk(z)

        if h==h_:
            return true
        else:
            return false

In [452]:
rain = RAIN()
m = 'hello'

(sk,pk) = rain.keygen()

z = rain.sign(sk,m)

rain.ver(m,z,pk)

TypeError: unsupported operand type(s) for -: 'bytes' and 'sage.modules.free_module_element.FreeModuleElement_generic_dense'